### UCF101 action recognition with C3D and Linear SVM
Experiment based on paper: https://arxiv.org/abs/1412.0767

Model implementation and pretrained weights is coming from this repo: https://github.com/SilvioGiancola/SoccerNet-code

Sports-1M dataset is described here: https://cs.stanford.edu/people/karpathy/deepvideo/deepvideo_cvpr2014.pdf

UCF101 dataset that was used as a benchmark for this experiment can be found here: https://arxiv.org/abs/1212.0402

In [41]:
%run ../python/data_prep.py
%run ../python/mpypl_pipe_func.py
%run ../python/mpypl_pipes.py

In [42]:
import pickle

TEST_SPLIT = get_test_split('../data/ucf101/testlist03.txt')
CLASSES = get_classes('../data/ucf101/classInd.txt')
MEAN_STD = pickle.load(open('../data/ucf101/mean_std.pkl', 'rb'))
DATA_DIR = '../data/ucf101/videos'

### STEP 1: Load all frames and save to file
Run only if you have full UCF101 under DATA_DIR and you wish to run it through the pipeline to extract frames from videos

In [ ]:
videos_to_frames_pipe(
    DATA_DIR, 
    ext='.avi',
    target_ext='.allframes.npy', 
    classes=CLASSES, 
    min_size=128, 
    max_elements=13320)

### STEP 2: Generate features for all frames from each video and average the resulting vectors
Run only if you have all video frames for UCF101 saved into separate .npy files and you want to run cropping, preprocessing and inferencing steps to generate and save feature vectors for each video 

In [54]:
# C3D_model function
%run ../python/c3dmodel.py

MODEL = get_video_descriptor(weights_path='../models/weights_C3D_sports1M_tf.h5')

Loading Model Weights from ../models/weights_C3D_sports1M_tf.h5
Popping last 3 layers


In [ ]:
frames_to_features_pipe(
    DATA_DIR,
    MEAN_STD,
    ext='.allframes.npy', 
    target_ext='.proc.c3d-avg.npy',
    model=MODEL,
    classes=CLASSES, 
    frames_per_clip=16, 
    frames_step=8, 
    batch_size=32, 
    max_elements=13320)

### STEP 3: Get features from saved files

In [45]:
data = get_features_from_files(
    data_dir=DATA_DIR, 
    features_ext='.proc.c3d-avg.npy', 
    test_split=[test_file.replace('.avi', '.proc.c3d-avg.npy') for test_file in TEST_SPLIT], 
    classes=CLASSES, 
    max_elements=13320)

........................................


In [48]:
X_train = np.asarray([x['c3d_avg'] for x in data if x['split']==mp.SplitType.Train])
Y_train = np.asarray([y['class_id'] for y in data if y['split']==mp.SplitType.Train], dtype=np.uint8)
X_test = np.asarray([x['c3d_avg'] for x in data if x['split']==mp.SplitType.Test])
Y_test = np.asarray([y['class_id'] for y in data if y['split']==mp.SplitType.Test], dtype=np.uint8)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(9624, 4096) (9624,)
(3696, 4096) (3696,)


In [49]:
%run ../python/pca_svm.py

### SVM+PCA10

In [50]:
run_pca_svm(X_train, Y_train, X_test, Y_test, n_components=10, labels=list(CLASSES.keys()))

Train/test shapes after PCA: (9624, 10) / (3696, 10)
                    precision    recall  f1-score   support

    ApplyEyeMakeup       0.34      0.68      0.45        37
     ApplyLipstick       0.55      0.82      0.66        33
           Archery       0.26      0.18      0.22        38
      BabyCrawling       0.48      0.72      0.58        39
       BalanceBeam       0.79      0.48      0.60        31
      BandMarching       0.67      0.86      0.76        43
     BaseballPitch       0.51      0.70      0.59        40
        Basketball       0.53      0.45      0.49        42
    BasketballDunk       0.80      0.97      0.88        40
        BenchPress       0.44      0.90      0.59        42
            Biking       0.73      0.75      0.74        40
         Billiards       0.95      1.00      0.98        42
       BlowDryHair       0.21      0.18      0.19        34
    BlowingCandles       0.35      0.39      0.37        28
  BodyWeightSquats       0.26      0.30      0

array([43, 18, 79, ..., 91, 32, 91], dtype=uint8)

### SVM+PCA50

In [51]:
run_pca_svm(X_train, Y_train, X_test, Y_test, n_components=50, labels=list(CLASSES.keys()))

Train/test shapes after PCA: (9624, 50) / (3696, 50)
                    precision    recall  f1-score   support

    ApplyEyeMakeup       0.55      0.73      0.63        37
     ApplyLipstick       0.79      0.70      0.74        33
           Archery       0.47      0.53      0.49        38
      BabyCrawling       0.87      0.85      0.86        39
       BalanceBeam       1.00      0.74      0.85        31
      BandMarching       0.77      0.93      0.84        43
     BaseballPitch       0.70      0.70      0.70        40
        Basketball       0.87      0.81      0.84        42
    BasketballDunk       1.00      0.97      0.99        40
        BenchPress       0.83      0.90      0.86        42
            Biking       0.87      1.00      0.93        40
         Billiards       1.00      1.00      1.00        42
       BlowDryHair       0.44      0.59      0.51        34
    BlowingCandles       0.59      0.79      0.68        28
  BodyWeightSquats       0.58      0.47      0

array([101,  27,  24, ...,  91,  32,  91], dtype=uint8)

### SVM+PCA100

In [52]:
run_pca_svm(X_train, Y_train, X_test, Y_test, n_components=100, labels=list(CLASSES.keys()))

Train/test shapes after PCA: (9624, 100) / (3696, 100)
                    precision    recall  f1-score   support

    ApplyEyeMakeup       0.61      0.68      0.64        37
     ApplyLipstick       0.86      0.76      0.81        33
           Archery       0.73      0.50      0.59        38
      BabyCrawling       0.84      0.82      0.83        39
       BalanceBeam       0.96      0.84      0.90        31
      BandMarching       0.87      0.95      0.91        43
     BaseballPitch       0.81      0.72      0.76        40
        Basketball       0.95      0.98      0.96        42
    BasketballDunk       1.00      0.97      0.99        40
        BenchPress       0.86      0.90      0.88        42
            Biking       0.95      1.00      0.98        40
         Billiards       1.00      1.00      1.00        42
       BlowDryHair       0.37      0.56      0.44        34
    BlowingCandles       0.62      0.89      0.74        28
  BodyWeightSquats       0.50      0.47     

array([101,  35,  24, ...,  91,  32,  91], dtype=uint8)

### SVM+PCA500

In [53]:
run_pca_svm(X_train, Y_train, X_test, Y_test, n_components=500, labels=list(CLASSES.keys()))

Train/test shapes after PCA: (9624, 500) / (3696, 500)
                    precision    recall  f1-score   support

    ApplyEyeMakeup       0.62      0.70      0.66        37
     ApplyLipstick       0.85      0.67      0.75        33
           Archery       0.75      0.55      0.64        38
      BabyCrawling       0.77      0.92      0.84        39
       BalanceBeam       0.96      0.84      0.90        31
      BandMarching       0.91      0.95      0.93        43
     BaseballPitch       0.72      0.78      0.75        40
        Basketball       0.87      0.98      0.92        42
    BasketballDunk       1.00      0.97      0.99        40
        BenchPress       0.93      0.90      0.92        42
            Biking       0.95      1.00      0.98        40
         Billiards       1.00      1.00      1.00        42
       BlowDryHair       0.43      0.68      0.52        34
    BlowingCandles       0.77      0.86      0.81        28
  BodyWeightSquats       0.48      0.47     

array([101,  35,  24, ...,  91,  32,  91], dtype=uint8)